In [8]:
import torch
import pandas as pd
import numpy as np
from src.transformers import BertTokenizer, BertForSequenceClassification,Trainer, TrainingArguments
from sklearn.metrics import classification_report
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [9]:
andmed = pd.read_csv("Rubric_data/estonianvalence.csv", encoding = "utf8", on_bad_lines='skip', header = None, 
                     names = ["rubric","url", "order", "sentiment", "text"])

In [17]:
andmed

,rubric,url,order,sentiment,text,rubric_idx
0,ARVAMUS,http://arvamus.postimees.ee/1001520/anvar-samost-tahan-politseile-kindel-olla/,1,negatiivne,"Enam kui kümme aastat tagasi tegutses huumorisaates «Wremja» inspektor Kukeke, kes kogu aeg vingus väikese palga pärast ja vaatas, mida saaks töö juurest koju tassida. Stsenaristid Andrus Kivirähk ja Mart Juur olid Kukekese isikusse kokku valanud kõik, mis 1990. aastate Eesti politseinikke halvast küljest iseloomustas.",0
1,ARVAMUS,http://arvamus.postimees.ee/1001520/anvar-samost-tahan-politseile-kindel-olla/,2,vastuoluline,"Neid ridu kirjutades tundub isegi ebaviisakas seda karikatuurset kuju meenutada. Juba saate eetrisse mineku ajal oli tegemist vaatega ajalukku. Politsei on vabanenud üleminekuajal parema puudumisel palgatud juhuseotsijatest. Samuti jõudis ühena esimestest politseijuhtideni arusaamine, et avalik kaeblemine palkade üle ei tule ühelegi organisatsioonile kasuks – kannatab maine ja raha juurde ei tule.",0
2,ARVAMUS,http://arvamus.postimees.ee/1001520/anvar-samost-tahan-politseile-kindel-olla/,3,positiivne,"Isiklikult kohtasin natukegi Kukekese moodi politseinikku viimati kaheksa aasta eest Lätis. Eranditult kõik viimase kümnendi kokkupuuted politseiametnikega on kinnitanud: vaatamata raskustele on Eesti riik suutnud korrakaitsjateks värvata inimesi, kes on arukad, kohusetundlikud, lugupidamist sisendavas füüsilises vormis ja hea väljendusoskusega.",0
3,ARVAMUS,http://arvamus.postimees.ee/1001520/anvar-samost-tahan-politseile-kindel-olla/,4,vastuoluline,"Olen näinud ka, kuidas patrull korrarikkujat taltsutab, ning suur osa sellest seisnes enesekindlas olekus ning vastuvaidlemist välistavalt, kuid rahulikult antud korraldustes. Aprillirahutuste ajal veendusime, et Eesti politsei suudab käituda ründajatega väga karmilt. Vaevalt et mitmeks tunniks näoli porisele asfaldile pandud märatsejat lohutas, et kohus hiljem teda riigijuhtide lubatud viisil aastateks vangi ei pannud.",0
4,ARVAMUS,http://arvamus.postimees.ee/1001520/anvar-samost-tahan-politseile-kindel-olla/,5,negatiivne,"Kummaline on nüüd äkki lugeda politsei ja siseministeeriumi ametnike kurtmist kohtu poolt politseinike ründajatele (lihtsalt öeldes – peksjatele) määratud karistuste üle. Tsiteerin ERRile intervjuu andnud politseinikku: «See ei ole normaalne, et sellised jõmikad tulevad, tümitavad ja nii ongi noh. Mingi tagajärg peab saabuma neile, [kes politseiametniku vastu kätt tõstavad].»",0
...,...,...,...,...,...,...
3843,VÄLISMAA,http://www.postimees.ee/998158/madridis-protestisid-tuhanded-inimesed-karpemeetmete-vastu/,3,negatiivne,"Hispaania peaminister Mariano Rajoy on Euroopa Liidu üha kasvava surve all, et ta vähendaks sel aastal eelarvepuudujääki 6,3 protsendini sisemajanduse koguproduktist (SKP), järgmisel aastal 4,5-ni ning 2014. aastal 2,8 protsendini. Kolme aastaga kavatseb valitsus hoida kokku 150 miljardit eurot.",2
3844,VÄLISMAA,http://www.postimees.ee/998158/madridis-protestisid-tuhanded-inimesed-karpemeetmete-vastu/,4,negatiivne,"Hispaania keskpank on hoiatanud, et riik ei pruugi sel aastal eelarvepuudujääki kavandatud mahus vähendada ning võib libiseda järgmisel aastal sügavasse kriisi.",2
3845,VÄLISMAA,http://www.postimees.ee/998270/ivanisvili-teeb-teatavaks-ministrikandidaadid/,1,neutraalne,"Gruusia parlamendivalimised võitnud koalitsiooni Gruusia Unistus liider, tulevane peaminister Bidzina Ivanišvili teeb esmaspäeval pressikonverentsil teatavaks ministrikandidaadid, ütles ta pühapäeval Facebookis.",2
3846,VÄLISMAA,http://www.postimees.ee/998270/ivanisvili-teeb-teatavaks-ministrikandidaadid/,2,neutraalne,Gruusia meedia on juba spekuleerinud tulevase valitsuse koosseisu üle.,2


In [11]:
andmed.rubric.value_counts()

ARVAMUS         972
KOMM-P-EESTI    487
KOMM-O-ELU      480
EESTI           413
SPORT           383
ELU-O           329
VÄLISMAA        313
KULTUUR         262
KRIMI           209
Name: rubric, dtype: int64

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Mappimine klasside indeksiteks ja tagasi
rubriigid = list(set(andmed["rubric"]))
rubriik_idx = {tag:idx for idx, tag in enumerate(rubriigid)}
idx_rubriik = {idx:tag for idx, tag in enumerate(rubriigid)}
andmed["rubric_idx"] = andmed["rubric"].map(rubriik_idx)

# Train 70%, Val 10%, Test 20%
train, test = train_test_split(andmed, test_size=0.2, stratify=andmed[['rubric_idx']])
train, val = train_test_split(train, test_size=0.125, stratify=train[['rubric_idx']])

tokenizer = BertTokenizer(vocab_file = "vocab_final.txt", vocab_file_form = "vocab_form.txt", max_length = 128,
                         padding = "max_length", truncation = True, return_tensors = "pt", mask_token="ˇMASKˇ")

In [13]:
print(train.rubric.value_counts())
print(val.rubric.value_counts())
print(test.rubric.value_counts())

ARVAMUS         680
KOMM-P-EESTI    341
KOMM-O-ELU      336
EESTI           289
SPORT           268
ELU-O           230
VÄLISMAA        219
KULTUUR         184
KRIMI           146
Name: rubric, dtype: int64
ARVAMUS         98
KOMM-P-EESTI    49
KOMM-O-ELU      48
EESTI           41
SPORT           38
ELU-O           33
VÄLISMAA        31
KULTUUR         26
KRIMI           21
Name: rubric, dtype: int64
ARVAMUS         194
KOMM-P-EESTI     97
KOMM-O-ELU       96
EESTI            83
SPORT            77
ELU-O            66
VÄLISMAA         63
KULTUUR          52
KRIMI            42
Name: rubric, dtype: int64


In [14]:
%%time
# Andmete loomine mudeli jaoks
train_encodings = tokenizer(list(train[:200].text), max_length = 128, padding = "max_length", 
                            truncation = True, return_tensors = "pt")
train_encodings["labels"] = train[:200].rubric_idx
train_dataset = Dataset.from_dict(train_encodings)

val_encodings = tokenizer(list(val[:200].text), max_length = 128, padding = "max_length", 
                            truncation = True, return_tensors = "pt")
val_encodings["labels"] = val[:200].rubric_idx
val_dataset = Dataset.from_dict(val_encodings)

test_encodings = tokenizer(list(test[:200].text), max_length = 128, padding = "max_length", 
                            truncation = True, return_tensors = "pt")
test_encodings["labels"] = test[:200].rubric_idx
test_dataset = Dataset.from_dict(test_encodings)

CPU times: total: 17.9 s
Wall time: 18.3 s


In [15]:
# Mudeli kohandamine
model = BertForSequenceClassification.from_pretrained("train_results/checkpoint-100000", num_labels=len(rubriik_idx))
model.to(device)

batch_size = 16

results_collection = []

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    predictions = [idx_rubriik[pred] for pred in predictions]
    labels = [idx_rubriik[lab] for lab in labels]

    results = classification_report(predictions, labels, output_dict=True)
    results_collection.append(results)
    return {"precision": results['weighted avg']['precision'], "recall": results['weighted avg']['recall'], "f1": results['weighted avg']['f1-score'], "accuracy": results["accuracy"]}
    

args = TrainingArguments(
    "Rubric_class_results",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=2
)


trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset= val_dataset,
    compute_metrics = compute_metrics
)

trainer.train()


Some weights of the model checkpoint at train_results_mudel4/checkpoint-200000 were not used when initializing BertForSequenceClassification: ['cls_lemma.predictions.transform.dense.bias', 'cls_lemma.predictions.bias', 'cls_lemma.predictions.transform.LayerNorm.bias', 'cls_lemma.predictions.transform.LayerNorm.weight', 'cls_lemma.predictions.transform.dense.weight', 'cls_lemma.predictions.decoder.bias', 'cls_lemma.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initi

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.946861,0.821227,0.305000,0.418326,0.305000
2,No log,1.835420,0.721806,0.385000,0.480766,0.385000


***** Running Evaluation *****
  Num examples = 200
  Batch size = 16
C:\Users\rauln\anaconda3\envs\EKTP\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rauln\anaconda3\envs\EKTP\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rauln\anaconda3\envs\EKTP\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg

TrainOutput(global_step=26, training_loss=1.9466438293457031, metrics={'train_runtime': 335.9926, 'train_samples_per_second': 1.191, 'train_steps_per_second': 0.077, 'total_flos': 52625518387200.0, 'train_loss': 1.9466438293457031, 'epoch': 2.0})

In [16]:
predictions, labels, _ = trainer.predict(test_dataset)
predictions = np.argmax(predictions, axis=1)

predictions = [idx_rubriik[pred] for pred in predictions]
labels = [idx_rubriik[lab] for lab in labels]

results = classification_report(predictions, labels, output_dict=True)
print(results)

***** Running Prediction *****
  Num examples = 200
  Batch size = 16


C:\Users\rauln\anaconda3\envs\EKTP\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rauln\anaconda3\envs\EKTP\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rauln\anaconda3\envs\EKTP\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'ARVAMUS': {'precision': 0.9130434782608695, 'recall': 0.34146341463414637, 'f1-score': 0.4970414201183432, 'support': 123}, 'EESTI': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 0}, 'ELU-O': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 0}, 'KOMM-O-ELU': {'precision': 0.04, 'recall': 1.0, 'f1-score': 0.07692307692307693, 'support': 1}, 'KOMM-P-EESTI': {'precision': 0.4482758620689655, 'recall': 0.22413793103448276, 'f1-score': 0.2988505747126437, 'support': 58}, 'KRIMI': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 0}, 'KULTUUR': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 0}, 'SPORT': {'precision': 0.5454545454545454, 'recall': 0.46153846153846156, 'f1-score': 0.4999999999999999, 'support': 13}, 'VÄLISMAA': {'precision': 0.16666666666666666, 'recall': 0.6, 'f1-score': 0.2608695652173913, 'support': 5}, 'accuracy': 0.325, 'macro avg': {'precision': 0.2348267280501163, 'recall': 0.29190442302301006, 'f1-score': 0.1815

C:\Users\rauln\anaconda3\envs\EKTP\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rauln\anaconda3\envs\EKTP\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rauln\anaconda3\envs\EKTP\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [39]:
### ESTBERT ###

tokenizer = AutoTokenizer.from_pretrained("tartuNLP/EstBERT", max_length = 128,
                         padding = "max_length", truncation = True, return_tensors = "pt")
train_encodings = tokenizer(list(train[:200].text), max_length = 128, padding = "max_length", 
                            truncation = True, return_tensors = "pt")
train_encodings["labels"] = train[:200].rubric_idx
train_dataset = Dataset.from_dict(train_encodings)

val_encodings = tokenizer(list(val[:200].text), max_length = 128, padding = "max_length", 
                            truncation = True, return_tensors = "pt")
val_encodings["labels"] = val[:200].rubric_idx
val_dataset = Dataset.from_dict(val_encodings)

test_encodings = tokenizer(list(test[:200].text), max_length = 128, padding = "max_length", 
                            truncation = True, return_tensors = "pt")
test_encodings["labels"] = test[:200].rubric_idx
test_dataset = Dataset.from_dict(test_encodings)

In [40]:
model = AutoModelForSequenceClassification.from_pretrained("tartuNLP/EstBERT", num_labels=len(rubriik_idx))
model.to(device)

batch_size = 16

results_collection = []

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    predictions = [idx_rubriik[pred] for pred in predictions]
    labels = [idx_rubriik[lab] for lab in labels]

    results = classification_report(predictions, labels, output_dict=True)
    results_collection.append(results)
    return {"precision": results['weighted avg']['precision'], "recall": results['weighted avg']['recall'], "f1": results['weighted avg']['f1-score'], "accuracy": results["accuracy"]}
    

args = TrainingArguments(
    "Rubric_class_results_EST",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=2
)


trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics = compute_metrics
)

trainer.train()

Some weights of the model checkpoint at tartuNLP/EstBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at tartuNLP/EstBERT and are new

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.890342,0.924461,0.270000,0.403413,0.270000
2,No log,1.804132,0.860580,0.330000,0.448330,0.330000


***** Running Evaluation *****
  Num examples = 200
  Batch size = 16
C:\Users\rauln\anaconda3\envs\EKTP\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rauln\anaconda3\envs\EKTP\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rauln\anaconda3\envs\EKTP\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg

TrainOutput(global_step=26, training_loss=1.8534036783071666, metrics={'train_runtime': 501.173, 'train_samples_per_second': 0.798, 'train_steps_per_second': 0.052, 'total_flos': 26312759193600.0, 'train_loss': 1.8534036783071666, 'epoch': 2.0})

In [41]:
predictions, labels, _ = trainer.predict(test_dataset)
predictions = np.argmax(predictions, axis=1)

predictions = [idx_rubriik[pred] for pred in predictions]
labels = [idx_rubriik[lab] for lab in labels]

results = classification_report(predictions, labels, output_dict=True)
print(results)

***** Running Prediction *****
  Num examples = 200
  Batch size = 16


C:\Users\rauln\anaconda3\envs\EKTP\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rauln\anaconda3\envs\EKTP\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rauln\anaconda3\envs\EKTP\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

     ARVAMUS       0.98      0.32      0.48       148
       EESTI       0.11      0.14      0.12        21
       ELU-O       0.00      0.00      0.00         0
  KOMM-O-ELU       0.22      0.28      0.24        18
KOMM-P-EESTI       0.14      0.50      0.21         6
       KRIMI       0.00      0.00      0.00         0
     KULTUUR       0.00      0.00      0.00         0
       SPORT       0.39      1.00      0.56         7
    VÄLISMAA       0.00      0.00      0.00         0

    accuracy                           0.33       200
   macro avg       0.20      0.25      0.18       200
weighted avg       0.77      0.33      0.42       200



C:\Users\rauln\anaconda3\envs\EKTP\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rauln\anaconda3\envs\EKTP\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rauln\anaconda3\envs\EKTP\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
